In [1]:
# download nltk stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# Install a particular version of `google-cloud-storage` because (oddly enough) 
# the  version on Colab and GCP is old. A dependency error below is okay.
!pip install -q google-cloud-storage==1.43.0

In [3]:
# authenticate below for Google Storage access as needed
from google.colab import auth
auth.authenticate_user()

In [5]:
# install ngrok to emulate public IP / address
!wget -N https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip

for details.

--2023-01-13 14:20:08--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 54.161.241.46, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  12.6MB/s    in 1.0s    

2023-01-13 14:20:10 (12.6 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [6]:
# TODO: sign up for an ngrok account
# then put your ngrok token below, uncomment, and execute
!./ngrok authtoken 22KeqcMQYpd2iRnsaZZOzlyaNw0_5NAk69pXRUMvfrf1oCNpU

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
# install a ngrok python package and a version of flask that works with it in 
# colab
!pip -q install flask-ngrok
!pip -q install flask==0.12.2
# !pip -q install flask_restful

     |████████████████████████████████| 83 kB 916 kB/s 


# Run the app

In [ ]:
# you need to upload your implementation of search_app.py
import search_frontend as se

In [ ]:
# uncomment the code below and execute to reload the module when you make 
# changes to search_frontend.py (after you upload again).
# import importlib
# importlib.reload(se)

In [ ]:
from flask_ngrok import run_with_ngrok
run_with_ngrok(se.app) 
se.app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://c271-34-125-136-123.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [04/Jan/2022 08:48:13] "GET /search?query=hello HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2022 08:48:13] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [04/Jan/2022 08:49:10] "GET /search_body?query=hello HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2022 08:49:10] "GET /favicon.ico HTTP/1.1" 404 -


# Testing your app

Once your app is running you can query it. You can simply do that by navigating to the URL that ngrok gave you above or through code in a different python session. For example, once the frontend app is running, you can navigate to:
http://YOUR_SERVER_DOMAIN/search?query=hello+world where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io, which is printed above in Colab or that is your external IP on GCP.

The code below shows how to issue a query from python. This is also how our testing code will issue queries to your search engine, so make sure to test your search engine this way after you deploy it to GCP and before submission. Command line instructions for deploying your search engine to GCP are available at `run_frontend_in_gcp.sh`. Note that we will not only issue training queries to your search engine, but also test queries, i.e. queries that you've never seen before.

In [2]:
map40 = {}  #all the
#dict:  k = query: v= [[recalls],[precision],avg_precision]

In [10]:
#recall = good/len(expected)
#precision = good/place
def map40_calculate(query, expected, predicted):#map 40 for query
  map40[query] = [[],[],0]
  predicted40 = predicted[:40]
  length_expected = len(expected)
  good = 0
  place = 0
  sum_precision = 0
  for p in predicted40:
    place += 1
    if p in expected:
      good+=1
      recall = good/length_expected
      precision = good/place
      map40[query][0].append(recall)
      map40[query][1].append(precision)
      sum_precision += precision
  #add the avg for the precision of the query
  avg_precision = sum_precision / len(map40[query][1])
  map40[query][2] = avg_precision



length = list(map40.items())
def avg_map40():
  sum_a = 0
  for k,v in map40.items():
    sum_a += v[2]
    print(sum_a)
  avg = sum_a / len(length)
  return avg





In [4]:
import json

with open('queries_train.json', 'rt') as f:
  queries = json.load(f)

In [5]:
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):        
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

In [6]:
import requests
from time import time
url = 'http://34.134.211.3:8080'
# place the domain you got from ngrok or GCP IP below. 
# url = 'http://XXXX-XX-XX-XX-XX.ngrok.io'

qs_res = []
arr = list(queries.items())
for i in range(len(arr)-15):
  q, true_wids = arr[i][0],arr[i][1]
# for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search', {'query': q}, timeout=35)
    duration = time() - t_start
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      ap = average_precision(true_wids, pred_wids)
      #our method map 40
      map40_calculate(q,true_wids, pred_wids)
      #################
      print("AP: ", ap)
  except:
    pass
  
  qs_res.append((q, duration, ap))

print(map40)



AP:  0.162
AP:  0.25
AP:  0.936
AP:  0.872
AP:  0.135
AP:  0.757
AP:  0.182
AP:  0.341
AP:  0.07
AP:  0.789
AP:  0.749
AP:  0.609
AP:  0.221
AP:  1.0
AP:  0.33
{'best marvel movie': [[0.022222222222222223, 0.044444444444444446, 0.06666666666666667, 0.08888888888888889, 0.1111111111111111, 0.13333333333333333, 0.15555555555555556, 0.17777777777777778, 0.2], [0.06666666666666667, 0.11764705882352941, 0.13043478260869565, 0.13793103448275862, 0.15625, 0.18181818181818182, 0.20588235294117646, 0.22857142857142856, 0.23076923076923078], 0.16177452629796313], 'How do kids come to world?': [[0.02], [0.25], 0.25], 'Information retrieval': [[0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0.175, 0.2, 0.225, 0.25, 0.275, 0.3, 0.325, 0.35, 0.375, 0.4, 0.425, 0.45, 0.475, 0.5, 0.525, 0.55, 0.575, 0.6], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9411764705882353, 0.8947368421052632, 0.8571428571428571, 0.8636363636363636, 0.8333333333333334, 0.7777777777777778, 0.7586206896

In [18]:
arr = list(map40.items())
length = len(list(map40.items()))
print("lenght: ", length)

def avg_map40():
  sum_a = 0
  for k,v in arr:
    sum_a += v[2]
  print("sum: ", sum_a)
  avg = sum_a / length
  return avg

print("AVG MAP40: " , avg_map40())

lenght:  15
sum:  7.40228698256728
AVG MAP40:  0.4934857988378187


In [20]:
import pandas as pd

new = pd.DataFrame.from_dict(map40)
new.to_csv("summary.csv")
new

,best marvel movie,How do kids come to world?,Information retrieval,LinkedIn,How to make coffee?,Ritalin,How to make wine at home?,Most expensive city in the world,India,how to make money fast?,Netflix,Apple computer,The Simpsons,World cup,How to lose weight?
0,"[0.022222222222222223, 0.044444444444444446, 0...",[0.02],"[0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0.175, ...","[0.06666666666666667, 0.13333333333333333, 0.2...","[0.021739130434782608, 0.043478260869565216, 0...","[0.02631578947368421, 0.05263157894736842, 0.0...","[0.02, 0.04, 0.06]","[0.021739130434782608, 0.043478260869565216, 0...","[0.02040816326530612, 0.04081632653061224]","[0.030303030303030304, 0.06060606060606061, 0....","[0.029411764705882353, 0.058823529411764705, 0...","[0.023809523809523808, 0.047619047619047616, 0...","[0.02564102564102564, 0.05128205128205128, 0.0...",[0.022727272727272728],"[0.022727272727272728, 0.045454545454545456, 0..."
1,"[0.06666666666666667, 0.11764705882352941, 0.1...",[0.25],"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.125, 0.14285714285714285, 0.125, 0.14814814...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.875, 0.888888...","[0.25, 0.16666666666666666, 0.13043478260869565]","[1.0, 0.18181818181818182, 0.08108108108108109...","[0.07142857142857142, 0.06896551724137931]","[1.0, 1.0, 1.0, 1.0, 0.35714285714285715, 0.375]","[1.0, 1.0, 1.0, 0.8, 0.8333333333333334, 0.545...","[1.0, 1.0, 0.6, 0.5714285714285714, 0.55555555...","[0.1111111111111111, 0.2, 0.2, 0.2352941176470...",[1.0],"[0.3333333333333333, 0.4, 0.42857142857142855,..."
2,0.161775,0.25,0.936137,0.871799,0.135251,0.756601,0.182367,0.340725,0.070197,0.78869,0.748742,0.609442,0.220722,1.0,0.329839


In [ ]:
import matplotlib.pyplot as plt

queries = map(lambda x:)
df = pd.DataFrame([[5, 2, 0], [3, 3, 2], [0, 1, 11]], classes, classes)
new.plot(kind = 'scatter', x = 'Duration', y = 'Calories')

In [25]:
print(qs_res[13])

('World cup', 4.538382053375244, 1.0)


In [4]:
sum = 0
for q, duration, ap in qs_res:
  sum+=ap

print(sum/len(qs_res))

0.4935333333333334
